In [1]:
import pandas as pd
import numpy as np
import pickle

dataframes = pd.read_pickle('recom_data/user_category_dummy6.pkl')

df_counts = dataframes['view_counts']
df_users = dataframes['users']

with open('recom_data/category_dict.pkl', 'rb') as handle:
    category_dict = pickle.load(handle)

item_similarity = pd.read_pickle('recom_data/item_similarity5.pkl')

# IBCF에서 사용할 유저 dummy데이터입니다.
db_user_data = [[1, 'F', 26]]
df_db_user_data = pd.DataFrame(db_user_data, columns=['user', 'gender', 'age'])
new_df_users = df_users.append(df_db_user_data)

# 이 유저는 1, 2, 3, 4, 31, 32, 33번 카테고리의 제품을 봤네요..
db_view_data = [[1, 1, 4], [1, 2, 2], [1, 3, 1], [1, 4, 3], [1, 31, 1], [1, 32, 1]]
df_db_view_data = pd.DataFrame(db_view_data, columns=['user', 'category', 'count'])
new_df_counts = df_counts.append(df_db_view_data)

view_matrix = new_df_counts.pivot(index='user', columns='category', values='count')

from sklearn.model_selection import train_test_split
x = new_df_counts.copy()
y = new_df_counts['user']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [2]:
print(new_df_counts)
print(new_df_users)

     user  category  count
0   90001        11      1
1   90001        12      1
2   90001        42      1
3   90001        43      1
4   90001        72      0
..    ...       ...    ...
1       1         2      2
2       1         3      1
3       1         4      3
4       1        31      1
5       1        32      1

[60016 rows x 3 columns]
       user gender  age
0     90141      M   20
1     90142      M   20
2     90143      M   20
3     90144      M   20
4     90145      M   20
...     ...    ...  ...
1136  91027      C  100
1137  91028      C  100
1138  91029      C  100
1139  91030      C  100
0         1      F   26

[1141 rows x 3 columns]


In [ ]:
# for category in view_matrix:
#     test = item_similarity[category].sort_values(ascending=False)[:10].reset_index()
#     test['category'] = test['category'].apply(lambda x: category_dict[x])
#     print(category_dict[category])
#     print(test)

In [ ]:
def recom_category(user_id, n_items=20, neighbor_size=28):
    user_category = view_matrix.loc[user_id].copy()
#     print(user_category)
    for category in view_matrix:
        user_category.loc[category] = IBCF_model(user_id, category, neighbor_size)
#     print(user_category)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    return category_sort

def IBCF_model(user, category, neighbor_size):
    sim_scores = item_similarity[category].copy()
    user_viewing = view_matrix.loc[user].copy()
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)
    
    neighbor_size = min(neighbor_size, len(sim_scores))
    sim_scores = np.array(sim_scores)
    user_viewing = np.array(user_viewing)
    user_idx = np.argsort(sim_scores)
    sim_scores = sim_scores[user_idx][-neighbor_size:]
    user_viewing = user_viewing[user_idx][-neighbor_size:]

    pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    
    return pred_viewing


# test = 90501
# pred_user_category = recom_category(user_id=test, n_items=10)
# recommend_category = pd.DataFrame(pred_user_category).reset_index()
# recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
# print(recommend_category)

test = 1
user_category = view_matrix.loc[test].copy()
source_category = view_matrix.loc[test].copy()

pred_user_category = recom_category(user_id=test, n_items=10, neighbor_size=40)

recommend_category = pd.DataFrame(pred_user_category).reset_index()
recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
print(test)
print(recommend_category)

In [ ]:
def recom_category(user_id, n_items=20, neighbor_size=30):
    for category in view_matrix:
        user_category.loc[category] = IBCF_model(user_id, category, neighbor_size)
    category_sort = user_category.sort_values(ascending=False)[:n_items]
    return category_sort

def IBCF_model(user, category, neighbor_size):
    sim_scores = item_similarity[category]
#     sim_scores[category] = 0
    user_viewing = source_category.T
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)
    if sim_scores.sum() >= 1.0:
#         print(category, sim_scores)
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        user_viewing = np.array(user_viewing)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        user_viewing = user_viewing[user_idx][-neighbor_size:]
        pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    else:
        pred_viewing = 0
                
    return pred_viewing


test = 90001
user_category = view_matrix.loc[test].copy()
source_category = view_matrix.loc[test].copy()

pred_user_category = recom_category(user_id=test, n_items=10, neighbor_size=40)

recommend_category = pd.DataFrame(pred_user_category).reset_index()
recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
print(test)
print(recommend_category)

# for test in range(90001, 90461, 20):
#     user_category = view_matrix.loc[test].copy()
#     source_category = view_matrix.loc[test].copy()

#     pred_user_category = recom_category(user_id=test, n_items=10, neighbor_size=40)

#     recommend_category = pd.DataFrame(pred_user_category).reset_index()
#     recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
#     print(test)
#     print(recommend_category)
    

# for test in range(90501, 90961, 20):
#     user_category = view_matrix.loc[test].copy()
#     source_category = view_matrix.loc[test].copy()

#     pred_user_category = recom_category(user_id=test, n_items=10, neighbor_size=40)

#     recommend_category = pd.DataFrame(pred_user_category).reset_index()
#     recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
#     print(test)
#     print(recommend_category)
    
# for test in range(91001, 91221, 20):
#     user_category = view_matrix.loc[test].copy()
#     source_category = view_matrix.loc[test].copy()

#     pred_user_category = recom_category(user_id=test, n_items=10, neighbor_size=40)

#     recommend_category = pd.DataFrame(pred_user_category).reset_index()
#     recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
#     print(test)
#     print(recommend_category)
    

In [ ]:
    test = 1
    user_category = view_matrix.loc[test].copy()
    source_category = view_matrix.loc[test].copy()

    pred_user_category = recom_category(user_id=test, n_items=10, neighbor_size=40)

    recommend_category = pd.DataFrame(pred_user_category).reset_index()
    recommend_category['category'] = recommend_category['category'].map(lambda x: category_dict[x])
    print(test)
    print(recommend_category)

In [ ]:
# RMSE 계산해주는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true)- np.array(y_pred)) ** 2))

# 모델별 RMSE 계산
def score(model, neighbor_size=30):
    id_pairs = zip(x_test['user'], x_test['category'])
    y_pred = np.array([model(user, category, neighbor_size) for (user, category) in id_pairs])
    y_true = np.array(x_test['count'])
    return RMSE(y_true, y_pred)

train_view_matrix = x_train.pivot(index='user', columns='category', values='count')
view_matrix_t = np.transpose(train_view_matrix)

In [ ]:
def IBCF_model(user, category, neighbor_size):
    sim_scores = item_similarity[category]
    user_viewing = view_matrix_t[user]
    non_viewing_idx = user_viewing[user_viewing.isnull()].index
    user_viewing = user_viewing.dropna()
    sim_scores = sim_scores.drop(non_viewing_idx)

    if sim_scores.sum() >= 1.5:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        user_viewing = np.array(user_viewing)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        user_viewing = user_viewing[user_idx][-neighbor_size:]
        pred_viewing = np.dot(sim_scores, user_viewing) / sim_scores.sum()
    else:
        pred_viewing = 0
    
    return pred_viewing

# print(score(IBCF_model, neighbor_size=423))

In [ ]:
# print(score(IBCF_model, neighbor_size=10))
# print(score(IBCF_model, neighbor_size=20))
# print(score(IBCF_model, neighbor_size=30))
# print(score(IBCF_model, neighbor_size=40))
# print(score(IBCF_model, neighbor_size=50))
# print(score(IBCF_model, neighbor_size=60))
# print(score(IBCF_model, neighbor_size=70))
# print(score(IBCF_model, neighbor_size=80))